In [1]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as np
import seaborn as sb
import xarray as xr
import os.path as path
from aux.ml_flood_config import path_to_data

In [2]:
# define some vars
data_path = f'{path_to_data}usa/monthly_files/'
print(data_path)

/home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/usa/monthly_files/


## rename files if necessary

In [3]:
from aux.utils import rename_files

#rename_files(path=data_path, old='day.', new='dayavg.', str_constraint='temperature')

## convert hourly files to daily means

In [4]:
from aux.utils import cdo_daily_means

#cdo_daily_means(path=data_path, file_includes='single_level')
#cdo_daily_means(path=data_path, file_includes='850_700_500')

## convert precipitation values to daily sums

In [ ]:
from aux.utils import cdo_precip_sums

#cdo_precip_sums(path=data_path, file_includes='large_scale_precipitation')

## extract precipitation parameters from daily sums and delete them from daily averages

In [ ]:
from aux.utils import cdo_clean_precip

#cdo_clean_precip(path=data_path)

## merge files (time dim) together

In [ ]:
from aux.utils import cdo_merge_time

# danube
#cdo_merge_time(path=data_path, file_includes='large_scale_precipitation', new_file='era5_lsp_1981-2017_daysum.nc')
#cdo_merge_time(path=data_path, file_includes='convective_precipitation', new_file='era5_tp_cp_1981-2017_daysum.nc')
#cdo_merge_time(path=data_path, file_includes='_temperature_', new_file='era5_z_t_q_1981-2017_dayavg.nc')
#cdo_merge_time(path=data_path, file_includes='soil_water', new_file='era5_swvl1_swvl2_ro_tcwv_1981-2017_dayavg.nc')

# usa
#cdo_merge_time(path=data_path, file_includes='geopotential,temperature', new_file='era5_z_t_q_1981-2017_dayavg.nc')
#cdo_merge_time(path=data_path, file_includes='convective_precipitation', new_file='era5_lsp_cp_1981-2017_daysum.nc')
#cdo_merge_time(path=data_path, file_includes='runoff', new_file='era5_ro_1981-2017_daysum.nc')
#cdo_merge_time(path=data_path, file_includes='soil_water', new_file='era5_swvl1_swvl2_tcwv_1981-2017_dayavg.nc')

# asia
#cdo_merge_time(path=data_path, file_includes='geopotential,temperature', new_file='era5_z_t_q_1981-2017_dayavg.nc')
#cdo_merge_time(path=data_path, file_includes='convective_precipitation', new_file='era5_lsp_cp_1981-2017_daysum.nc')
#cdo_merge_time(path=data_path, file_includes='runoff', new_file='era5_ro_1981-2017_daysum.nc')
#cdo_merge_time(path=data_path, file_includes='soil_water', new_file='era5_swvl1_swvl2_tcwv_1981-2017_dayavg.nc')

# GLOFAS

## 1981-2002 done; todo: 2003-2018

In [ ]:
from aux.utils import cdo_spatial_cut
# define some vars
data_path = f'{path_to_data}glofas/'
print(data_path)
#!ls /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/


# extract data from tar (in terminal)
#tar -xvf yourfile.tar

# DANUBE
# cut out spatial region (needs to be added to utils file)
cdo_spatial_cut(path=data_path, file_includes='dis_', new_file_includes='danube', lonmin=7, lonmax=20, latmin=47, latmax=50)
# merge into one file (already exists in utils)
cdo_merge_time(path=data_path, file_includes='spatial_cut_danube', new_file='glofas_reanalysis_danube_1981-2002.nc')

# USA
cdo_spatial_cut(path=data_path, file_includes='dis_', new_file_includes='usa', lonmin=-125, lonmax=-70, latmin=25, latmax=50)
cdo_merge_time(path=data_path, file_includes='spatial_cut_usa', new_file='glofas_reanalysis_usa_1981-2002.nc')

# ASIA
cdo_spatial_cut(path=data_path, file_includes='dis_', new_file_includes='asia', lonmin=35, lonmax=140, latmin=0, latmax=55)
cdo_merge_time(path=data_path, file_includes='spatial_cut_asia', new_file='glofas_reanalysis_asia_1981-2002.nc')

/home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/
extracting region: dis_19810101_19810201.nc to dis_19810101_19810201_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19820101_19820201.nc to dis_19820101_19820201_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19830101_19830201.nc to dis_19830101_19830201_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19840101_19840201.nc to dis_19840101_19840201_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19850101_19850201.nc to dis_19850101_19850201_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19860101_19860201.nc to dis_19860101_19860201_spatial_cut_danube.nc in /home/srvx11/lehre/users/a130358

extracting region: dis_19880301_19880401.nc to dis_19880301_19880401_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19890301_19890401.nc to dis_19890301_19890401_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19900301_19900401.nc to dis_19900301_19900401_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19910301_19910401.nc to dis_19910301_19910401_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19920301_19920401.nc to dis_19920301_19920401_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19930301_19930401.nc to dis_19930301_19930401_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19940

extracting region: dis_19950501_19950601.nc to dis_19950501_19950601_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19960501_19960601.nc to dis_19960501_19960601_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19970501_19970601.nc to dis_19970501_19970601_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19980501_19980601.nc to dis_19980501_19980601_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19990501_19990601.nc to dis_19990501_19990601_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_20000501_20000601.nc to dis_20000501_20000601_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_20010

extracting region: dis_20020701_20020801.nc to dis_20020701_20020801_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19810801_19810901.nc to dis_19810801_19810901_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19820801_19820901.nc to dis_19820801_19820901_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19830801_19830901.nc to dis_19830801_19830901_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19840801_19840901.nc to dis_19840801_19840901_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19850801_19850901.nc to dis_19850801_19850901_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19860

extracting region: dis_19871001_19871101.nc to dis_19871001_19871101_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19881001_19881101.nc to dis_19881001_19881101_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19891001_19891101.nc to dis_19891001_19891101_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19901001_19901101.nc to dis_19901001_19901101_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19911001_19911101.nc to dis_19911001_19911101_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19921001_19921101.nc to dis_19921001_19921101_spatial_cut_danube.nc in /home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/glofas/ ...
extracting region: dis_19931